In [3]:
import csv
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
#function that loads a lexicon of positive words to a set and returns the set
def loadLexicon(fname):
    newLex=set()
    lex_conn=open(fname)

    #add every word in the file to the set
    for line in lex_conn:
        newLex.add(line.strip())# remember to strip to remove the lin-change character
    lex_conn.close()

    return newLex


In [52]:
def Parse(input_file, index1, index2):
    # Load the positive and negative lexicons into sets
    posLex = loadLexicon('positive-words.txt')
    negLex = loadLexicon('negative-words.txt')

    noun_sentiment_index1 = {}  # Maps each noun to a tuple (positive count, negative count)
    noun_sentiment_index2 = {}

    fin = open(input_file, encoding='utf8', newline='')
    reader = csv.reader(fin)

    for i, line in enumerate(reader):
        if i not in [index1]:  # Skip rows except for row 23
            continue

        text, rating = line  # Get the text and rating
        sentences = sent_tokenize(text)  # Split the review into sentences

        for sentence in sentences:  # For each sentence
            words = word_tokenize(sentence)  # Split the sentence into words
            tagged_words = nltk.pos_tag(words)  # Perform POS tagging on the words in the sentence

            nouns_in_sentence = set()  # Set of all the nouns in the sentence
            positive_word_count = 0  # Number of positive words in the sentence
            negative_word_count = 0  # Number of negative words in the sentence

            # Iterate over the tagged words
            for tagged_word in tagged_words:
                if tagged_word[1].startswith('NN'):  # If the word is a noun
                    noun = tagged_word[0].lower()  # Lowercase the noun

                    if len(noun) < 3:  # Ignore nouns with less than 3 characters
                        continue

                    nouns_in_sentence.add(noun)  # Add the noun to the set

                if tagged_word[1].startswith('JJ'):  # If the word is an adjective
                    if tagged_word[0] in posLex:  # Check if it is a positive word
                        positive_word_count += 1
                    elif tagged_word[0] in negLex:  # Check if it is a negative word
                        negative_word_count += 1

            for noun in nouns_in_sentence:  # For each noun found in the sentence
                noun_sentiment_index1[noun] = (
                    noun_sentiment_index1.get(noun, (0, 0))[0] + positive_word_count,
                    noun_sentiment_index1.get(noun, (0, 0))[1] + negative_word_count
                )

    sorted_nouns_index1 = sorted(noun_sentiment_index1.items(), key=lambda x: sum(x[1]), reverse=True)

    filtered_result_1 = [(noun, sentiment) for noun, sentiment in sorted_nouns_index1 if sentiment != (0, 0)]

    filtered_result_1 = [(noun, 'Positive' if sentiment[0] > sentiment[1] else 'Negative' if sentiment[0] < sentiment[1] else 'Neutral') for noun, sentiment in filtered_result_1]



    fin.seek(0)  # Reset the file position to the beginning

    for i, line in enumerate(reader):
        if i not in [index2]:  # Skip rows except for row 23
            continue

        text, rating = line  # Get the text and rating
        sentences = sent_tokenize(text)  # Split the review into sentences

        for sentence in sentences:  # For each sentence
            words = word_tokenize(sentence)  # Split the sentence into words
            tagged_words = nltk.pos_tag(words)  # Perform POS tagging on the words in the sentence

            nouns_in_sentence = set()  # Set of all the nouns in the sentence
            positive_word_count = 0  # Number of positive words in the sentence
            negative_word_count = 0  # Number of negative words in the sentence

            # Iterate over the tagged words
            for tagged_word in tagged_words:
                if tagged_word[1].startswith('NN'):  # If the word is a noun
                    noun = tagged_word[0].lower()  # Lowercase the noun

                    if len(noun) < 3:  # Ignore nouns with less than 3 characters
                        continue

                    nouns_in_sentence.add(noun)  # Add the noun to the set

                if tagged_word[1].startswith('JJ'):  # If the word is an adjective
                    if tagged_word[0] in posLex:  # Check if it is a positive word
                        positive_word_count += 1
                    elif tagged_word[0] in negLex:  # Check if it is a negative word
                        negative_word_count += 1

            for noun in nouns_in_sentence:  # For each noun found in the sentence
                noun_sentiment_index2[noun] = (
                    noun_sentiment_index2.get(noun, (0, 0))[0] + positive_word_count,
                    noun_sentiment_index2.get(noun, (0, 0))[1] + negative_word_count
                )

    sorted_nouns_index2 = sorted(noun_sentiment_index2.items(), key=lambda x: sum(x[1]), reverse=True)

    filtered_result_2 = [(noun, sentiment) for noun, sentiment in sorted_nouns_index2 if sentiment != (0, 0)]

    filtered_result_2 = [(noun, 'Positive' if sentiment[0] > sentiment[1] else 'Negative' if sentiment[0] < sentiment[1] else 'Neutral') for noun, sentiment in filtered_result_2]

    list_opposite_nouns = []
    for noun1, sentiment1 in filtered_result_1:
      for noun2, sentiment2 in filtered_result_2:
        if noun1 == noun2 and sentiment1 != sentiment2:
          print('Index1:', noun1, sentiment1)
          print('Index2:', noun2, sentiment2)
          list_opposite_nouns.append(noun1)

    print(list_opposite_nouns)
    print(filtered_result_1)
    print(filtered_result_2)


    fin.close()




In [53]:
result=Parse('amazonreviews.csv',36,50)
print(result)

Index1: sennheiser Positive
Index2: sennheiser Negative
['sennheiser']
[('sennheiser', 'Positive'), ('hd650', 'Positive'), ('headphones', 'Positive'), ('lot', 'Positive'), ('noise', 'Positive'), ('cancelation', 'Positive'), ('app', 'Negative'), ('tile', 'Negative')]
[('pair', 'Negative'), ('today', 'Neutral'), ('months', 'Neutral'), ('bluetooth', 'Neutral'), ('issues', 'Neutral'), ('side', 'Neutral'), ('disconnections', 'Neutral'), ('sennheiser', 'Negative')]
None
